In [0]:
import re

In [0]:
dbutils.widgets.text("file_name", "")
file_name = dbutils.widgets.get("file_name")

dbutils.widgets.text("year", "")
year = dbutils.widgets.get("year")

match = re.match(r"([^-]+)-bordereaux.*", file_name)
if match:
    if file_name.find(year) == -1:
        raise ValueError("Coverage year should match the file name")

    table_name = match.group(1)
else:
    raise ValueError("Invalid file name format. Expected format: <table>-bordereaux-<year>.csv")

excel_file_path = f"/mnt/bordereaux/bronze/{year}/{file_name}"
delta_table = f"hive_metastore.bordereaux.{table_name}_cleaned"

df = spark.read.csv(
    excel_file_path,
    header=True,
    inferSchema=False,
    multiLine=True,         # In case some fields have line breaks
    quote='"',              # Properly handle comma-separated quoted fields
    escape='"',             # Handle quotes inside quoted fields
    mode="PERMISSIVE"       # Handle malformed lines gracefully
)

# Then load data in the bordereaux delta table
df.write.format("delta") \
    .mode("append") \
    .option("mergeSchema", "true") \
    .saveAsTable(delta_table)